In [ ]:
import pandas as pd
import numpy as np
df_raw = pd.read_csv('./data/BTCUSDT_1h.csv')
df_raw.head()

In [ ]:
import numpy as np
# Initialize support_time column with NaN
df = df_raw.copy()
df['support_time'] = np.nan

# Calculate support_time
for i in range(len(df)):
    low = df.loc[i, 'low_px']
    for j in range(i - 1, -1, -1):
        if df.loc[j, 'low_px'] < low < df.loc[j, 'high_px']:
            df.loc[i, 'support_time'] = df.loc[j, 'open_time']
            break
df['support'] = df.apply(
    lambda row: df[df['open_time'] == row['support_time']]['low_px'].values[0]
    if not np.isnan(row['support_time']) else row['low_px'],
    axis=1
)
df['support_time_diff'] = df['open_time'] - df['support_time']
n_support_segment = df['support_time_diff'].sum() / (df['open_time'].iat[-1] - df['open_time'].iat[0]) + 1

In [ ]:
df['resistance_time'] = np.nan

# Calculate resistance_time
for i in range(len(df)):
    high = df.loc[i, 'high_px']
    for j in range(i - 1, -1, -1):
        if df.loc[j, 'low_px'] < high < df.loc[j, 'high_px']:
            df.loc[i, 'resistance_time'] = df.loc[j, 'open_time']
            break

df['resistance'] = df.apply(
    lambda row: df[df['open_time'] == row['resistance_time']]['high_px'].values[0]
    if not np.isnan(row['resistance_time']) else row['high_px'],
    axis=1
)
df['resistance_time_diff'] = df['open_time'] - df['resistance_time']
n_resistance_segment = df['resistance_time_diff'].sum() / (df['open_time'].iat[-1] - df['open_time'].iat[0]) + 1

In [ ]:
import mplfinance as mpf

symbol = df['symbol'].values[0]
timestamp = df['open_time'].values[0]

df_plot = df.copy()
df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
apds = [mpf.make_addplot(df['support'], type='line', color='green', linestyle='dashed'),
        mpf.make_addplot(df['resistance'], type='line', color='red', linestyle='dashed')]


# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)


In [ ]:
# Pivot(t) = High(t-1) + Low(t-1) + Close(t-1) / 3
# Support_1(t) = (2 * Pivot(t)) - High(t-1)
# Support_2(t) = Pivot(t) - (High(t-1) - Low(t-1))
df = df_raw.copy()
df['pivot'] = (df['high_px'].shift(1) + df['low_px'].shift(1) + df['close_px'].shift(1)) / 3
df['support_1'] = (2 * df['pivot']) - df['high_px'].shift(1)
df['support_2'] = df['pivot'] - (df['high_px'].shift(1) - df['low_px'].shift(1))

In [ ]:
import mplfinance as mpf

symbol = df['symbol'].values[0]
timestamp = df['open_time'].values[0]

df_plot = df.copy()
df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
apds = [mpf.make_addplot(df['pivot'], type='line', color='blue', linestyle='dashed'),
        mpf.make_addplot(df['support_1'], type='line', color='green', linestyle='dashed'),
        mpf.make_addplot(df['support_2'], type='line', color='red', linestyle='dashed')]


# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)

In [ ]:
import pwlf
df.sort_values(by='support_2', ascending=True, inplace=True)
df.dropna(inplace=True)
model = pwlf.PiecewiseLinFit(df['open_time'], df['support_2'])
break_pts = [df['open_time'].min(), df['open_time'].iat[0], df['open_time'].iat[-1], df['open_time'].max()]
break_pts = sorted(break_pts)
# print(break_pts)
model.fit_with_breaks(break_pts)
x_hat = df['open_time'].sort_values(ascending=True)
# Sort back to original order
df.sort_values(by='open_time', ascending=True, inplace=True)
df['support_model_1'] = model.predict(x_hat)



In [ ]:
import mplfinance as mpf

symbol = df['symbol'].values[0]
timestamp = df['open_time'].values[0]

df_plot = df.copy()
df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
apds = [mpf.make_addplot(df['support_model_1'], type='line', color='blue', linestyle='dashed'),       
        mpf.make_addplot(df['support_2'], type='line', color='red', linestyle='dashed')]


# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)

In [ ]:
# Import necessary libraries
from arch import arch_model
import matplotlib.pyplot as plt

df = df_raw.copy()
df['log_return'] = np.log(df['close_px'] / df['open_px'])

# Ensure there are no NaN values in the log_return column
df.dropna(subset=['log_return'], inplace=True)

# Fit a GARCH(1,1) model to the log returns
garch_model = arch_model(df['log_return'], vol='Garch', p=1, q=1, mean='Constant', dist='Normal')
garch_fit = garch_model.fit(disp='off')

# Print the summary of the model
print(garch_fit.summary())

# Plot the conditional volatility
plt.figure(figsize=(10, 6))
plt.plot(garch_fit.conditional_volatility, label='Conditional Volatility')
plt.title('Conditional Volatility from GARCH(1,1) Model')
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.show()

# Analyze the residuals
residuals = garch_fit.resid
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, alpha=0.75, label='Residuals')
plt.title('Histogram of Residuals')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()
